In [3]:
from urllib.request import urlopen
import pandas as pd
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

In [19]:
#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urlopen(wiki) 
soup = BeautifulSoup(page)
canda_table=soup.find('table', class_="wikitable sortable")
Postcode=[]
Borough=[]
Neighbourhood=[]
for row in canda_table.findAll("tr"):
    row_cells=row.find_all('td')
    if len(row_cells)==3:
        Postcode.append(row_cells[0].find(text=True))
        Borough.append(row_cells[1].find(text=True))
        Neighbourhood.append(row_cells[2].find(text=True))


df=pd.DataFrame()
df['Postcode']=Postcode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood
assigned_Borough=df[df.Borough!='Not assigned']
after_Merge = assigned_Borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
after_Merge=after_Merge.replace('Not assigned\n',after_Merge[after_Merge.Neighbourhood.str.contains('Not assigned\n')].Borough.to_string())
after_Merge.rename(columns={'Postcode':'Postal Code'},inplace=True)
after_Merge.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


<b> Using geocoder it is taking very long time so I have used the CSV file instead</b><br>
!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder


lat_lng_coords = None


while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M1B"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [21]:
latAndlongFromCSV=pd.read_csv("Geospatial_Coordinates.csv")
latAndlongFromCSV.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df3 = pd.merge(after_Merge, latAndlongFromCSV, how='outer', on=['Postal Code'])
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
